In [ ]:
def PNC_reg_summary_processing(year, time_diff):
    
    query =  f"""SELECT 
           DATEPART(year, pa.CreatedOn) as yyyy
           , DATEPART(month,pa.CreatedOn) as mm
           , min(DATEDIFF(minute, t.CreatedOn, t.ModifiedOn)) MinProcessTime
           , max(DATEDIFF(minute, t.CreatedOn, t.ModifiedOn)) MaxProcessTime
           , AVG(DATEDIFF(minute, t.CreatedOn, t.ModifiedOn)) AvgProcessTime
    --     , DATEPART(day, pa.CreatedOn) as dd
           , COUNT(*) AS cnt
    FROM
           stl_action pa with (nolock)
           INNER join stl_stltask t with (nolock) on pa.stl_actionId = t.stl_parentaction
                  and t.StatusCode NOT IN (279640001, 1)
                  AND DATEDIFF(minute, t.CreatedOn, t.ModifiedOn) < {time_diff}
    WHERE
           1=1
           AND pa.stl_name = 'PNCRegistration'
           AND t.[subject] = 'Send Registration To PNC'
           and DATEPART(year, pa.CreatedOn) > {year}
    GROUP BY
           DATEPART(year, pa.CreatedOn)
           , DATEPART(month,pa.CreatedOn)
    ORDER BY
           DATEPART(year, pa.CreatedOn) DESC
           , DATEPART(month, pa.CreatedOn)  DESC"""

    return query


In [ ]:
def PNC_reg_throughput(year):
    
    query =  f""";WITH DAILY AS
            (
            SELECT 
                   DATEPART(year, pa.CreatedOn) as yyyy
                   , DATEPART(month,pa.CreatedOn) as mm
                   , DATEPART(day, pa.CreatedOn) as dd
                   , COUNT(*) AS cnt
            FROM
                   stl_action pa with (nolock)
            WHERE
                   1=1
                   and pa.stl_name = 'PNCRegistration'
                   and DATEPART(year, pa.CreatedOn) > {year}
                   --and DATEPART(month, pa.CreatedOn) > 01

            GROUP BY
                   DATEPART(year, pa.CreatedOn)
                   , DATEPART(month,pa.CreatedOn) 
                   , DATEPART(day,pa.CreatedOn) 

            )
            SELECT 
                   yyyy
                   , mm
                   , MIN(cnt) as dailyMin
                   , MAX(cnt) as dailyMax
                   , AVG(cnt) as dailyAvg
                   , sum(cnt) as totalForMonth
            FROM
                   DAILY
            group by
                   yyyy
                   , mm
            order by
                   yyyy desc
                   , mm desc
            """

    return query